# Expresiones regulares como fuente de datos faltantes
---

### Importo, preparo y cargo df

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_propierati = pd.read_csv('../properati.zip', compression='zip')

In [3]:
pd.set_option('display.max_colwidth', 1500)
pd.set_option('display.max_rows', 200)

## Preguntas iniciales
---

In [4]:
df_propierati.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [5]:
df_propierati.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

## Regex
---

Patrones

In [6]:
# Precios inmueble
precios_inmueble_pat = '(usd|u\$d|u\$s|us\$|dolares) ?(\d{2,}(\,|\.)?\d{0,3}(\,|\.)?d{0,3}(\,|\.)?\d{0,3})[^\w] ?(dolares)?(?!(.*m2|.*mc|.*metros))'
precios_inmueble_reg = re.compile(precios_inmueble_pat, flags = re.IGNORECASE)

# Metros inmueble
metros_inmueble_pat = '(\d{1,4}\,?\.?\d{0,4}) ?(m2|mc|metros|mts)'
metros_inmueble_reg = re.compile(metros_inmueble_pat, flags = re.IGNORECASE)

# Ambientes - hay propiedaddes de 20 ambientes o mas?
#amb_pat = '(\d{1,2})(?= ambientes)'
amb_pat = '(?<!\-) ([1-2][0-9]?)(?= amb[^o])' # version mejorada
amb_reg = re.compile(amb_pat, flags = re.IGNORECASE)

# Cochera
cochera_pat = '(?<!sin )(?<!no incluye )(?<!no posee )cochera(?! no incluye)(?! no incluida)'
cochera_reg = re.compile(cochera_pat, flags = re.IGNORECASE)

# Piscina/Pileta
piscina_pat = 'piscina|pileta(?! de baño)(?! de cocina)'
piscina_reg = re.compile(piscina_pat, flags = re.IGNORECASE)

# Parilla
parrilla_pat = '(?<!sin )(?<!no incluye )(?<!no posee )parrilla(?! no incluye)(?! no incluida)'
parrilla_reg = re.compile(parrilla_pat, flags = re.IGNORECASE)

# Baulera
baulera_pat = '(?<!sin )(?<!no incluye )(?<!no posee )Baulera(?! no incluye)(?! no incluida)'
baulera_reg = re.compile(baulera_pat, flags = re.IGNORECASE)

## Regex Precios
---

In [47]:
df_propierati.price_aprox_usd.isnull().value_counts()

False    100810
True      20410
Name: price_aprox_usd, dtype: int64

Campo Titulo

In [48]:
df_propierati['precios_normal_regex_titulo'] = df_propierati.title.apply(lambda x: x if x is np.NaN else precios_inmueble_reg.search(x))
df_propierati['precios_normal_regex_titulo'] = df_propierati.precios_normal_regex_titulo \
    .apply(lambda x: 0 if x is None else x.group(2) \
           .replace('.','') \
           .replace(',','')) \
    .astype(int)

In [49]:
df_propierati['precios_normal_regex_titulo'].notnull().value_counts()

True    121220
Name: precios_normal_regex_titulo, dtype: int64

In [50]:
# analisis de outliers para identificar que eliminar
precios_normal_regex_titulo = df_propierati.precios_normal_regex_titulo.apply(lambda x: x if x > 30000 and x < 35000000 else np.NaN)
df_propierati['precios_normal_regex_titulo'] = df_propierati.precios_normal_regex_titulo.apply(lambda x: x if x > 30000 and x < 35000000 else np.NaN)

In [51]:
df_propierati['precios_normal_regex_titulo'].notnull().value_counts()

False    118601
True       2619
Name: precios_normal_regex_titulo, dtype: int64

Campo Descripcion

In [52]:
df_propierati['precios_normal_regex_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else precios_inmueble_reg.search(x))
df_propierati['precios_normal_regex_desc'] = df_propierati.precios_normal_regex_desc \
    .apply(lambda x: 0 if x is None or x is np.NaN else x.group(2) \
           .replace('.','') \
           .replace(',','')) \
    .astype(int)

In [53]:
df_propierati['precios_normal_regex_desc'].notnull().value_counts()

True    121220
Name: precios_normal_regex_desc, dtype: int64

In [54]:
precios_normal_regex_desc = df_propierati.precios_normal_regex_desc.apply(lambda x: x if x > 30000 and x < 20000000 else np.NaN)
df_propierati['precios_normal_regex_desc'] = df_propierati.precios_normal_regex_desc.apply(lambda x: x if x > 30000 and x < 20000000 else np.NaN)

In [55]:
df_propierati['precios_normal_regex_desc'].notnull().value_counts()

False    115327
True       5893
Name: precios_normal_regex_desc, dtype: int64

In [56]:
print(df_propierati['precios_normal_regex_titulo'].apply(lambda x: type(x)).value_counts(), '\n', df_propierati['precios_normal_regex_titulo'].dtypes)

<class 'float'>    121220
Name: precios_normal_regex_titulo, dtype: int64 
 float64


In [57]:
print(df_propierati['precios_normal_regex_desc'].apply(lambda x: type(x)).value_counts(), '\n', df_propierati['precios_normal_regex_desc'].dtypes)

<class 'float'>    121220
Name: precios_normal_regex_desc, dtype: int64 
 float64


In [58]:
df_propierati['precios_normal_regex_desc'].size, df_propierati['precios_normal_regex_titulo'].size

(121220, 121220)

In [59]:
price_aprox_usd_null_mask = df_propierati.price_aprox_usd.isnull()

In [60]:
df_propierati.loc[price_aprox_usd_null_mask, 'price_aprox_usd'] = df_propierati.loc[price_aprox_usd_null_mask, 'precios_normal_regex_titulo']

In [61]:
df_propierati.price_aprox_usd.isnull().value_counts()

False    103188
True      18032
Name: price_aprox_usd, dtype: int64

In [62]:
price_aprox_usd_null_mask = df_propierati.price_aprox_usd.isnull()

In [63]:
df_propierati.loc[price_aprox_usd_null_mask, 'price_aprox_usd'] = df_propierati.loc[price_aprox_usd_null_mask, 'precios_normal_regex_desc']

In [64]:
df_propierati.price_aprox_usd.isnull().value_counts()

False    103310
True      17910
Name: price_aprox_usd, dtype: int64

## Regex ambientes
---

In [28]:
df_propierati.rooms.apply(lambda x: type(x)).value_counts()

<class 'float'>    121220
Name: rooms, dtype: int64

In [29]:
df_propierati_rooms_null_mask = df_propierati.rooms.isnull()
df_propierati.rooms.isnull().value_counts()

True     73830
False    47390
Name: rooms, dtype: int64

In [30]:
df_propierati['rooms_regex_titl'] = df_propierati.title.apply(lambda x: x if x is np.NaN else amb_reg.search(x))
df_propierati['rooms_regex_titl'] = df_propierati.rooms_regex_titl.apply(lambda x: x if x is None else x.group(1))
df_propierati['rooms_regex_titl'] = df_propierati.rooms_regex_titl.apply(lambda x: np.NaN if x is None else float(x))

In [31]:
df_propierati.rooms_regex_titl.isnull().value_counts()

True     114860
False      6360
Name: rooms_regex_titl, dtype: int64

In [32]:
df_propierati.rooms_regex_titl.value_counts()

2.0     5333
1.0     1019
11.0       3
10.0       2
15.0       1
14.0       1
12.0       1
Name: rooms_regex_titl, dtype: int64

In [33]:
df_propierati['rooms_regex_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else amb_reg.search(x))
df_propierati['rooms_regex_desc'] = df_propierati.rooms_regex_desc.apply(lambda x: x if x is np.NaN or x is None else x.group(1))
df_propierati['rooms_regex_desc'] = df_propierati.rooms_regex_desc.apply(lambda x: np.NaN if x is None else float(x))

In [34]:
df_propierati.rooms_regex_desc.isnull().value_counts()

True     106492
False     14728
Name: rooms_regex_desc, dtype: int64

In [35]:
df_propierati.rooms_regex_desc.value_counts()

2.0     12482
1.0      2210
10.0       13
11.0       12
18.0        3
14.0        3
12.0        2
17.0        1
25.0        1
22.0        1
Name: rooms_regex_desc, dtype: int64

In [36]:
df_propierati.loc[df_propierati_rooms_null_mask, 'rooms'] = df_propierati.loc[df_propierati_rooms_null_mask, 'rooms_regex_titl']
df_propierati.rooms.isnull().value_counts()

True     71455
False    49765
Name: rooms, dtype: int64

In [37]:
df_propierati_rooms_null_mask = df_propierati.rooms.isnull()

In [38]:
df_propierati.loc[df_propierati_rooms_null_mask, 'rooms'] = df_propierati.loc[df_propierati_rooms_null_mask, 'rooms_regex_desc']
df_propierati.rooms.isnull().value_counts()

True     65802
False    55418
Name: rooms, dtype: int64

## Regex Metros
---

In [7]:
df_propierati.surface_total_in_m2.apply(lambda x: type(x)).value_counts()

<class 'float'>    121220
Name: surface_total_in_m2, dtype: int64

In [8]:
df_propierati.surface_total_in_m2.isnull().value_counts()

False    81892
True     39328
Name: surface_total_in_m2, dtype: int64

In [9]:
# analisis de outliers para identificar que eliminar

df_propierati['totalm2_normal_regex_titulo'] = df_propierati.title.apply(lambda x: x if x is np.NaN else metros_inmueble_reg.search(x))
df_propierati['totalm2_normal_regex_titulo'] = df_propierati.totalm2_normal_regex_titulo.apply(lambda x: x if x is None else x.group(1))

#df_propierati['totalm2_normal_regex_titulo'] = df_propierati.totalm2_normal_regex_titulo.apply(lambda x: x if x is None else x.group(1).replace(',','.'))

#df_propierati['totalm2_normal_regex_titulo'] = df_propierati.totalm2_normal_regex_titulo.apply(lambda x: 0 if x is np.NaN else x)
#f_propierati['totalm2_normal_regex_titulo'] = df_propierati['totalm2_normal_regex_titulo'].astype(int)

#df_propierati.loc[55106, 'totalm2_normal_regex_titulo']
#df_propierati.loc[17768, ['totalm2_normal_regex_titulo', 'title']]


df_propierati.surface_total_in_m2.value_counts()
#pepe = df_propierati.totalm2_normal_regex_titulo.apply(lambda x: np.NaN if x is None else float(x))
#df_propierati.totalm2_normal_regex_titulo.value_counts()
#pepe.sort_values(ascending=True).head(100)
#df_propierati['totalm2_normal_regex_titulo'].apply(lambda x: type(x)).value_counts()
#df_propierati['totalm2_normal_regex_titulo'].isnull().value_counts()
df_propierati['totalm2_normal_regex_titulo'].sort_values(ascending=True).head(200)
#df_propierati['totalm2_normal_regex_titulo'].value_counts()

27520          1
56822          1
56828          1
32984      1.000
14561      1.000
68526      1.000
68497      1.000
68477      1.000
11124      1.013
68583      1.100
66342      1.100
69007      1.133
65183      1.144
64804      1.200
66425      1.200
65122      1.230
64808      1.255
65791      1.270
97448      1.300
66298      1.340
64901      1.400
65270      1.500
97611      1.500
96283      1.500
66012      1.622
68727      1.707
115251     1.720
65845      1.750
65532      1.800
67016      1.890
97524      1.892
65095      1.950
73961         10
81026         10
22474         10
7592          10
33510         10
42575         10
82953         10
30576         10
69115     10.960
42665        100
7873         100
80921        100
51234        100
96776        100
24622        100
51527        100
80777        100
52324        100
32090        100
80538        100
52338        100
51147        100
6501         100
78009        100
42532        100
52820        100
53344        1

In [78]:
s = pd.Series(['98.5', '49.00', '85.74', np.NaN])
s = s.astype(float)
s

0    98.50
1    49.00
2    85.74
3      NaN
dtype: float64

---

## Regex amenities
---

### Cocheras

Titulo

In [11]:
df_propierati['cochera_regex_titulo'] = df_propierati.title.apply(lambda x: x if x is np.NaN else cochera_reg.search(x))
df_propierati['cochera_regex_titulo'] = df_propierati.cochera_regex_titulo.apply(lambda x: x if x is None else x.group())

In [13]:
cochera_regex_titulo_notnull_mask = df_propierati['cochera_regex_titulo'].notnull()
df_propierati.loc[cochera_regex_titulo_notnull_mask, 'cochera_regex_titulo'] = 1
cochera_regex_titulo_null_mask = df_propierati['cochera_regex_titulo'].isnull()
df_propierati.loc[cochera_regex_titulo_null_mask, 'cochera_regex_titulo'] = np.NaN

df_propierati['cochera_regex_titulo'].isnull().value_counts()

True     115235
False      5985
Name: cochera_regex_titulo, dtype: int64

Descripcion

In [14]:
df_propierati['cochera_regex_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else cochera_reg.search(x))
df_propierati['cochera_regex_desc'] = df_propierati.cochera_regex_desc.apply(lambda x: x if x is None or x is np.NaN else x.group())

In [15]:
cochera_regex_desc_notnull_mask = df_propierati['cochera_regex_desc'].notnull()
df_propierati.loc[cochera_regex_desc_notnull_mask, 'cochera_regex_desc'] = 1
cochera_regex_desc_null_mask = df_propierati['cochera_regex_desc'].isnull()
df_propierati.loc[cochera_regex_desc_null_mask, 'cochera_regex_desc'] = np.NaN

df_propierati['cochera_regex_desc'].isnull().value_counts()

True     79062
False    42158
Name: cochera_regex_desc, dtype: int64

In [17]:
df_propierati['cochera'] = df_propierati['cochera_regex_titulo']
df_propierati.loc[df_propierati['cochera_regex_titulo'].isnull(), 'cochera'] = df_propierati.loc[df_propierati['cochera_regex_titulo'].isnull(), 'cochera_regex_desc']
df_propierati['cochera'].isnull().value_counts()

True     78075
False    43145
Name: cochera, dtype: int64

### Piscinas

Titulo

In [18]:
df_propierati['piscina_regex_titulo'] = df_propierati.title.apply(lambda x: x if x is np.NaN else piscina_reg.search(x))
df_propierati['piscina_regex_titulo'] = df_propierati.piscina_regex_titulo.apply(lambda x: x if x is None else x.group())

In [19]:
piscina_regex_titulo_notnull_mask = df_propierati['piscina_regex_titulo'].notnull()
df_propierati.loc[piscina_regex_titulo_notnull_mask, 'piscina_regex_titulo'] = 1
piscina_regex_titulo_null_mask = df_propierati['piscina_regex_titulo'].isnull()
df_propierati.loc[piscina_regex_titulo_null_mask, 'piscina_regex_titulo'] = np.NaN

df_propierati['piscina_regex_titulo'].isnull().value_counts()

True     119616
False      1604
Name: piscina_regex_titulo, dtype: int64

Descripcion

In [20]:
df_propierati['piscina_regex_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else piscina_reg.search(x))
df_propierati['piscina_regex_desc'] = df_propierati.piscina_regex_desc.apply(lambda x: x if x is None or x is np.NaN else x.group())

In [21]:
piscina_regex_desc_notnull_mask = df_propierati['piscina_regex_desc'].notnull()
df_propierati.loc[piscina_regex_desc_notnull_mask, 'piscina_regex_desc'] = 1
piscina_regex_desc_null_mask = df_propierati['piscina_regex_desc'].isnull()
df_propierati.loc[piscina_regex_desc_null_mask, 'piscina_regex_desc'] = np.NaN

df_propierati['piscina_regex_desc'].isnull().value_counts()

True     88487
False    32733
Name: piscina_regex_desc, dtype: int64

In [22]:
df_propierati['piscina'] = df_propierati['piscina_regex_titulo']
df_propierati.loc[df_propierati['piscina_regex_titulo'].isnull(), 'piscina'] = df_propierati.loc[df_propierati['piscina_regex_titulo'].isnull(), 'piscina_regex_desc']
df_propierati['cochera'].isnull().value_counts()

True     78075
False    43145
Name: cochera, dtype: int64

### Parilla

Titulo

In [23]:
df_propierati['parrilla_regex_titulo'] = df_propierati.title.apply(lambda x: x if x is np.NaN else parrilla_reg.search(x))
df_propierati['parrilla_regex_titulo'] = df_propierati.parrilla_regex_titulo.apply(lambda x: x if x is None else x.group())

In [24]:
parrilla_regex_titulo_notnull_mask = df_propierati['parrilla_regex_titulo'].notnull()
df_propierati.loc[parrilla_regex_titulo_notnull_mask, 'parrilla_regex_titulo'] = 1
parrilla_regex_titulo_null_mask = df_propierati['parrilla_regex_titulo'].isnull()
df_propierati.loc[parrilla_regex_titulo_null_mask, 'parrilla_regex_titulo'] = np.NaN

df_propierati['parrilla_regex_titulo'].isnull().value_counts()

True     120548
False       672
Name: parrilla_regex_titulo, dtype: int64

Descripcion

In [25]:
df_propierati['parrilla_regex_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else parrilla_reg.search(x))
df_propierati['parrilla_regex_desc'] = df_propierati.parrilla_regex_desc.apply(lambda x: x if x is None or x is np.NaN else x.group())

In [26]:
parrilla_regex_desc_notnull_mask = df_propierati['parrilla_regex_desc'].notnull()
df_propierati.loc[parrilla_regex_desc_notnull_mask, 'parrilla_regex_desc'] = 1
parrilla_regex_desc_null_mask = df_propierati['parrilla_regex_desc'].isnull()
df_propierati.loc[parrilla_regex_desc_null_mask, 'parrilla_regex_desc'] = np.NaN

df_propierati['parrilla_regex_desc'].isnull().value_counts()

True     87187
False    34033
Name: parrilla_regex_desc, dtype: int64

In [27]:
df_propierati['parrilla'] = df_propierati['parrilla_regex_titulo']
df_propierati.loc[df_propierati['parrilla_regex_titulo'].isnull(), 'parrilla'] = df_propierati.loc[df_propierati['parrilla_regex_titulo'].isnull(), 'parrilla_regex_desc']
df_propierati['parrilla'].isnull().value_counts()

True     87129
False    34091
Name: parrilla, dtype: int64

### Baulera

Titulo

In [35]:
df_propierati['baulera_regex_titulo'] = df_propierati.title.apply(lambda x: x if x is np.NaN else baulera_reg.search(x))
df_propierati['baulera_regex_titulo'] = df_propierati.baulera_regex_titulo.apply(lambda x: x if x is None else x.group())

In [36]:
baulera_regex_titulo_notnull_mask = df_propierati['baulera_regex_titulo'].notnull()
df_propierati.loc[baulera_regex_titulo_notnull_mask, 'baulera_regex_titulo'] = 1
baulera_regex_titulo_null_mask = df_propierati['baulera_regex_titulo'].isnull()
df_propierati.loc[baulera_regex_titulo_null_mask, 'baulera_regex_titulo'] = np.NaN

df_propierati['baulera_regex_titulo'].isnull().value_counts()

True     120693
False       527
Name: baulera_regex_titulo, dtype: int64

Descripcion

In [37]:
df_propierati['baulera_regex_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else baulera_reg.search(x))
df_propierati['baulera_regex_desc'] = df_propierati.baulera_regex_desc.apply(lambda x: x if x is None or x is np.NaN else x.group())

In [38]:
baulera_regex_desc_notnull_mask = df_propierati['baulera_regex_desc'].notnull()
df_propierati.loc[baulera_regex_desc_notnull_mask, 'baulera_regex_desc'] = 1
baulera_regex_desc_null_mask = df_propierati['baulera_regex_desc'].isnull()
df_propierati.loc[baulera_regex_desc_null_mask, 'baulera_regex_desc'] = np.NaN

df_propierati['baulera_regex_desc'].isnull().value_counts()

True     110435
False     10785
Name: baulera_regex_desc, dtype: int64

In [39]:
df_propierati['baulera'] = df_propierati['baulera_regex_titulo']
df_propierati.loc[df_propierati['baulera_regex_titulo'].isnull(), 'baulera'] = df_propierati.loc[df_propierati['baulera_regex_titulo'].isnull(), 'baulera_regex_desc']
df_propierati['baulera'].isnull().value_counts()

True     110378
False     10842
Name: baulera, dtype: int64

---

In [40]:
df_propierati.head(10)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,cochera,piscina_regex_titulo,piscina_regex_desc,piscina,parrilla_regex_titulo,parrilla_regex_desc,parrilla,baulera_regex_titulo,baulera_regex_desc,baulera
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
6,6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente López|Munro|,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
9,9,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---

In [178]:
# conclusion, regex podria ser [1-9][0-9] en vez de \d{1,2}
df_propierati.description.loc[df_propierati.rooms_regex_desc == '02'].head(2)

25659    DEPARTAMENTOS PRE-VENTA DE POZO PARA 2017FINANCIADOS EN PESOSULTIMOS DISPONIBLES Studio 25 m2 con un anticipo de $335.300 y 48 cuotas de $7.9502 ambientes amplio con un anticipo de $721.350  y 48 cuotas de $17.175ANTICIPO FINANCIADO HASTA 12 CUOTAS.TODO EN PESOSSIN REQUISITOSEdificio de 4 pisosParking para bicicletas, puertas blindadas, caja de seguridad, cerraduras electrnicas, UF c/parrilla, portero visor, circuito interno TV, interiores placard, energa solar luz pasillos, parrilla general, laundry, terraza parquizada, aire acondicionado individual, SUM, solarium, jacuzzi, canchita de basquet, laundryConsltenosTe:        15 Mail: departamentos. / fede. .Publicado por edificios financiados  a traves INMOMAP
25662    DEPARTAMENTOS PRE-VENTA DE POZO PARA 2017FINANCIADOS EN PESOSULTIMOS DISPONIBLES Studio 25 m2 con un anticipo de $335.300 y 48 cuotas de $7.9502 ambientes amplio con un anticipo de $721.350  y 48 cuotas de $17.175ANTICIPO FINANCIADO HASTA 12 CUOTAS.TODO EN PESOSSI

In [71]:
df_propierati.description.loc[df_propierati.rooms_regex_desc == '25']

5403    Venta de Hotel  Tilisarao, San Luis, 16 Habitaciones 25 ambientes , 10 cocheras semicubiertas , quincho, GRAN OPORTUNIDAD DE INVERSION!! HOTEL s/Terreno: 1200 m2 Construcción: planta baja: 290m2 y 1er. Piso: 172,50 m2 Cocheras s/cubiertas: 10 vehículos.- Antigüedad: 30 años planta baja, ampliación 1er piso 10 años.- Todo lo que es conexiones eléctricas, de agua y cloacas fueron renovadas totalmente hace 8 años.- Cantidad Habitaciones: Total 16 (Planta Baja: 1 recepción, 1 comedor, 2 baños (hombre y mujer), 1 oficina privada, 1 cocina, 1 depósito de ropa blanca y planchado, 1 deposito chico, 8 habitaciones con baño privado, 1 sector lavadero y baño de servicio, 1 pieza de termo tanques, y pasillo central. 1er. Piso: 8 habitaciones con baño privado, pasillo central y depósito de ropa blanca y termotanque.- Otros sectores: Cocheras para 10 vehículos, depósito de cosas mayores, Quincho con asador, mesada y AA, Cisterna y bomba con 3000 litros de reserva de agua, Patio.- Posibles am

In [179]:
# esta pegado a otro valor numerico!!
df_propierati.description.loc[df_propierati.rooms_regex_desc == '93'].head(2)

49022    Corredor Responsable: Jorge Salafia - CUCICBA 5666Contacto: Viviana Tendler - MLS ID # 420211116-693 Ambientes  con cochera fija, y baulera individual 2 años de uso. Súper luminoso Piso 2, vista abierta al contra frente , muy silencioso Expensas $5000 a Junio 2017 incluye Cochera, Aysa, Edenor, ABL Piso de porcelanato simil madera Todo eléctrico Cocina semi integrada con espacio de guardado, revestimiento de venecita y meadas de granito. Espacio para lavarropas Closet de la entrada Balcón semi aterrazado. de  13 m2  Superficie cubierta 71,37 m2 (según Boleto) 2 habitaciones con placares con una hoja espejada Baño completo con revestimiento de venecita . Aire acondicionado  frio calor en todos los ambientes  Terraza con parrilla  Apto Profesional No es Apto crédito Primera escritura- cesión de Boleto  Espacio para bicicletas  A solo 500 metros de AV. Cabildo, 9 cuadras del tren estación Nuñez línea Mitre,10 cuadras de estación Congreso, Subte D.  
52452                         

In [180]:
df_propierati.description.loc[df_propierati.rooms_regex_desc == '15'].head(2)

47136    Corredor Responsable: Daniel Acosta - CUCICBA 2406Contacto: Francisco Coelho - MLS ID # 42015 ambientes , al frente, luminoso. de época, con balcón terraza.- 3 dormitorios, living, cocina comedor.-  Lavadero, 2 baños.-  Buena relación precio y ubicación.- Apto profesional.- A media cuadra de Av Segurola, a una de Gaona.- A reciclar, se escuchan ofertas.-   Expensas $3128  DANIEL ACOSTA CUCICBA 2406 / CSI 5953"En cumplimiento de la Ley 2340 CUCICBA, Ley 10.973 de la Prov.Bs.As., Ley Nacional 25.028, Ley 22.802 de Lealtad Comercial, Ley 24.240 de Defensa al Consumidor, las normas del Código Civil y Comercial de la Nación y Constitucionales, los agentes NO ejercen el corretaje inmobiliario. Todas las operaciones inmobiliarias son objeto de intermediacion y conclusión por parte de los martilleros y corredores colegiados, cuyos datos se exhiben debajo del nombre de la inmobiliaria." 
Name: description, dtype: object

In [181]:
df_propierati.title.loc[df_propierati.rooms_regex_titl == '14'].head(2)

112408    San Telmo ,Piso de 14 ambientes   Terraza! 400 Metros. Edificio Historico
Name: title, dtype: object

In [182]:
df_propierati.description.loc[df_propierati.rooms_regex_desc == '14'].head(2)

52223                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Corredor Responsable: Rene Alejandro Vera - CSI 5848Contacto: Ramiro Pesaresi - MLS ID # 420301054-14 ambientes con muy buena dimensión e iluminación. Solarium de 25m2 descubierto con pileta de natación. La casa se compone de living comedor cocina tipo americano (integrad

In [252]:
df_propierati.description.loc[1937]

'fqwefqwefqwef'

In [323]:
df_propierati.description.loc[76509]

'VENDE: LOBOSPROPIEDAD CÉNTRICA.CALLE BALCARCE 175.E/ MORENO Y ALBERDI.SUP. LOTE: 237,00 MTS.EDIFICADO: 112,00 MTS.DOS HABITACIONES.COCINA, COMEDOR DIARIO.BAÑO, LAVADERO, PATIO.GAS NATURAL, CLOACAS.AGUA CORRIENTE, ASFALTO.RM1, F.O.S: 0.60 / F.O.T: 2,00VALOR: U$S 185.000,00.CONSULTAR. -  - FERNANDO BLANCO INMOBILIARIA'

In [54]:
df_propierati.title.loc[90]

'EXCELENTE | MENDOZA 237 | MTS PLAZA COLO'

In [67]:
df_propierati.title.loc[106376]

'LAVALLOL. Predio Industrial de 93.000 m2 y 30.000 m2 construidos.1200 m2 de Oficinas. Unico en la Zona'

In [68]:
df_propierati.title.loc[64803]

'UNICA!!... CASA DE ESTILO ....con añeja arboleda..rodeada de un entorno espectacular!!!...9.500mts de parque..650mts de primerísima construcción'

In [71]:
df_propierati.title.loc[55106]

'Departamento 4 ambientes al frente con balcón corrido, 2 baños, baulera, 96,69 mts2.'

In [58]:
df_propierati.title.loc[17768]

'A MTS  FACULTAD DE DERECHO- LA PERLA - SOBRE 28 MTS . EQUIPADO .'

In [115]:
df_propierati.title.loc[38]

'Venta depto,loft, Cañitas,210 ms,pileta propia!,cochera,baulera'

In [116]:
df_propierati.title.loc[71]

'COMPLEJO DTOS. EN PH - DOS DTOS DE 3 AMB. CON COCHERA - LOS PINARES'

In [117]:
df_propierati.title.loc[70]

'COMPLEJO DE DTOS. EN PH. - TRES DTOS DE 2 AMB. CON COCHERA . LOS PINARES'

In [131]:
df_propierati.description.loc[101]

'Tucuman 3476Edificio PB y 4 pisos por ascensorDepartamentos de 1 dormitorio, 2 dormitorios, monoambientes y cocheras.Monoambientes de 22m2 + 5.30m21 Dormitorio al frente de 36m2 + 5.30m21 Dormitorio contrafrente de 34.50m2 + 2.30m22 Dormitorios al contrafrente de 52.50m2 + 2.70m2ENTREGA ENERO 2018Posibilidad de anticipo y cuotas hasta posesión. 4404242 / 4403232Santa Fe Piso 6 Oficina 42000 | Rosario | Santa Fe | Argentina Horario de Atención: Lunes a Viernes de 10.00 a 18.00. EasyBroker ID: EB-AI0783'

In [144]:
df_propierati.description.loc[16]

'Ventas Mataderos al frente Duplex 4 amb.- Planta Baja: Entrada de auto-Living Comedor- toilete, Cocina Comedor-diario- parque (4m x 10m) con parrilla. Planta Alta: 3 dormitorios-placard- 1 baño completo- Dormitorio principal en suite y vestidor. Todo luz y sol en excelente estado a metros de Av. J. B. Alberdi y Larrazabal ."Listo para vivir ya!!!Las medidas son aproximadas y al sólo efecto orientativo. Las medidas reales surgirán del título de propiedad. '

In [145]:
df_propierati.description.loc[78]

'S/AVDA   COSTANERA ,  NUEVA  ZONA GASTRONOMICA  FRENTE AL BALNEARIO EL BRETE. IDEAL VIVIENDA  FLIAR. MAS EMPRENDIMIENTO  COMERCIAL  APTO RESTAURANTE Y PARRILLA. PLANTA ALTA 3 DORM (1 EN SUITE)  PATIO FRENTE Y FONDO, $  5. 500.000.- SE RECIBE  INMUEBLE A CUENTA. TERRENO 24.14 x 10.78. CUENTA CON BAÑO PARA DISCAPACITADO, Y HABILITACION PARA COMERCIO GASTRONOMICO.'

# Pruebas sobre dataset propierati

---

Importacion de librerias necesarias.

In [1]:
import pandas as pd
import numpy as np
import re

Carga de datos y establecimiento de parametros de visualizacion.

In [2]:
df_propierati = pd.read_csv('../properati.zip', compression='zip')
df_prop_uz = pd.read_csv('../properati.csv')

In [3]:
pd.set_option('display.max_colwidth', 40)

### datosCol(serie) - Funcion que retorna diccionario con los datos de una Serie / Columna del DataFrame a analizar:

Campos:
 - nombre columna.
 - tipo de dato serie.
 - cantidad de registros en la serie.
 - cantidad de nulos en la serie.
 - cantidad de NO nulos en la serie.
 - porcentaje de No nulos en la serie.
 - cantidad de valores unicos en la serie.
 - valor mas repetido en la serie.
 - valor maximo en la serie.
 - valor minimos en la serie.
 - media, mediana y desvio estandar en la serie.
 - distribucion de cuantiles.
 - tipos de datos sobre los valores incluidos en la serie.

In [4]:
def datosCol(col=None):
    try:
        if col.dtype != object:
            return {
                    'nombre': col.name,
                    'series_dt': col.dtype,
                    'registros': int(len(col)), 
                    'nulos': int(len(col) - col.describe()[0]), 
                    'no_nulos': int(col.describe()[0]),
                    'no_nulos_pc': round(col.describe()[0] / col.shape[0], 4),
                    'unicos': int(len(col.value_counts())),
                    'mas_repetido': col.value_counts().index[0],
                    'minimo': round(col.min(), 2),
                    'maximo': round(col.max(), 2),
                    'media': round(col.describe()[1], 4),
                    'mediana': col.median(),
                    'desvio': round(col.describe()[2], 4),
                    'q25': col.describe()[4],
                    'q50': col.describe()[5],
                    'q75': col.describe()[6],
                    'values_dt': [x for x in col.apply(lambda x: type(x)).value_counts().index]
                   }
        else:
            return {
                    'nombre': col.name,
                    'series_dt': col.dtype,
                    'registros': int(col.describe()[0]),
                    'no_nulos': int(col.notnull().sum()),
                    'no_nulos_pc': round(col.notnull().sum() / col.shape[0], 4),
                    'unicos': int(col.describe()[1]), 
                    'mas_repetido': col.describe()[2],
                    'nulos': col.isnull().sum(),
                    'frecuencia': int(col.describe()[3]),
                    'values_dt': [x for x in col.apply(lambda x: type(x)).value_counts().index]
                   }
    except:
        return 'use a valid pandas Series'

In [5]:
df_info = pd.DataFrame.from_dict(datosCol(df_propierati.operation))
df_info = df_info.append(datosCol(df_propierati.property_type), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.place_name), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.place_with_parent_names), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.country_name), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.state_name), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.geonames_id), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati['lat-lon']), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.lat), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.lon), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.price), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.currency), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.price_aprox_local_currency), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.price_aprox_usd), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.surface_total_in_m2), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.surface_covered_in_m2), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.price_usd_per_m2), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.price_per_m2), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.floor), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.rooms), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.expenses), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.properati_url), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.title), ignore_index=True)
df_info = df_info.append(datosCol(df_propierati.description), ignore_index=True)

In [6]:
df_info

,nombre,series_dt,registros,no_nulos,no_nulos_pc,unicos,mas_repetido,nulos,frecuencia,values_dt,desvio,maximo,media,mediana,minimo,q25,q50,q75
0,operation,object,121220,121220,1.0000,1,sell,0,121220.0,<class 'str'>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,property_type,object,121220,121220,1.0000,4,apartment,0,71065.0,[<class 'str'>],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,place_name,object,121197,121197,0.9998,1060,Córdoba,23,9254.0,"[<class 'str'>, <class 'float'>]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,place_with_parent_names,object,121220,121220,1.0000,1164,|Argentina|Santa Fe|Rosario|,0,8504.0,[<class 'str'>],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,country_name,object,121220,121220,1.0000,1,Argentina,0,121220.0,[<class 'str'>],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,state_name,object,121220,121220,1.0000,28,Capital Federal,0,32316.0,[<class 'str'>],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,geonames_id,float64,121220,102503,0.8456,646,3.83857e+06,18717,NaN,[<class 'float'>],3.541306e+05,6.948895e+06,3.574442e+06,3.433910e+06,3427208.00,3.430234e+06,3.433910e+06,3.836668e+06
7,lat-lon,object,69670,69670,0.5747,47203,"-34.4026444,-58.6684776",51550,312.0,"[<class 'str'>, <class 'float'>]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,lat,float64,121220,69670,0.5747,46495,-34.4026,51550,NaN,[<class 'float'>],1.980900e+00,4.550000e+00,-3.462620e+01,-3.459799e+01,-54.82,-3.466907e+01,-3.459799e+01,-3.444130e+01
9,lon,float64,121220,69670,0.5747,46522,-58.6685,51550,NaN,[<class 'float'>],2.299900e+00,-5.373000e+01,-5.926630e+01,-5.848013e+01,-75.68,-5.872704e+01,-5.848013e+01,-5.839591e+01


In [7]:
total_m2_mask = df_propierati.surface_total_in_m2.notnull()
df_propierati.loc[total_m2_mask, :].max()
df_propierati.loc[total_m2_mask, 'geonames_id':].sort_values(by='surface_total_in_m2', ascending=True).head(5)

,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
6244,3838574.0,"-32.9343838,-60.6467153",-32.934384,-60.646715,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15td9_ve...,Ubicado en la Zona Centro de Rosario...,Departamento tipo loft a estrenar z...,https://thumbs4.properati.com/6/snx4...
14969,3430513.0,"-34.5460181,-58.6963849",-34.546018,-58.696385,454000.0,ARS,449263.9,25461.98,0.0,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/16toh_ve...,Emprendimiento de pozo complejo Olim...,"Duplex de pozo ""Complejo Olimpo""",https://thumbs4.properati.com/5/1YCl...
14037,3433359.0,"-34.663673,-58.562634",-34.663673,-58.562634,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/16qz2_ve...,CHALET COMODIDADES:PB.: amplio livin...,ITUZAINGO NORTE,https://thumbs4.properati.com/0/dIDO...
60348,3433359.0,"-34.663673,-58.562634",-34.663673,-58.562634,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19z3f_ve...,CHALETen planta bajaCOMODIDADES: liv...,ITUZAINGO NORTE,https://thumbs4.properati.com/7/5CD4...
14967,NaN,"-34.5367608,-58.7160784",-34.536761,-58.716078,250000.0,USD,4411125.0,250000.00,0.0,NaN,NaN,NaN,NaN,NaN,5000.0,http://www.properati.com.ar/16toc_ve...,Esplendido departamento de 5 ambient...,MAGNIFICO DEPTO DE 5 AMBIENTES - SEM...,https://thumbs4.properati.com/3/Qcb6...


In [8]:
# Tipos de datos columnas
df_propierati.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

In [9]:
# URLs no existen
df_propierati.properati_url[1]

'http://www.properati.com.ar/15bob_venta_departamentos_la-plata_balcon_lavadero_toilette_garage_estrenar_antonini-propiedades_dcp'

In [10]:
# Dimensiones data frame y tamañano
print(df_propierati.shape)
print(df_propierati.size)

(121220, 26)
3151720


In [11]:
# cantidad de registros
df_propierati.index

RangeIndex(start=0, stop=121220, step=1)

In [12]:
# cantidad de columnas
df_propierati.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [13]:
#df_propierati.title[0:10]

In [14]:
# Porcentaje de nulos por columna
df_propierati.isnull().sum() / df_propierati.shape[0]

Unnamed: 0                    0.000000
operation                     0.000000
property_type                 0.000000
place_name                    0.000190
place_with_parent_names       0.000000
country_name                  0.000000
state_name                    0.000000
geonames_id                   0.154405
lat-lon                       0.425260
lat                           0.425260
lon                           0.425260
price                         0.168372
currency                      0.168380
price_aprox_local_currency    0.168372
price_aprox_usd               0.168372
surface_total_in_m2           0.324435
surface_covered_in_m2         0.164222
price_usd_per_m2              0.433947
price_per_m2                  0.276869
floor                         0.934837
rooms                         0.609058
expenses                      0.882346
properati_url                 0.000000
description                   0.000016
title                         0.000000
image_thumbnail          

In [15]:
# Nulos por columna
df_propierati.shape[0] - df_propierati.notnull().sum()

Unnamed: 0                         0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

In [16]:
prop_floor_mask = df_propierati.floor.notnull()
df_propierati.loc[prop_floor_mask, :].head(2)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
83,83,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.954876705,-60.6518377614",-32.954877,...,55.0,NaN,NaN,5.0,2.0,NaN,http://www.properati.com.ar/15byz_ve...,IDEAL INVERSORES o ESTUDIANTES APTA ...,2 Dormitorios. Dorrego y Pellegrini,https://thumbs4.properati.com/5/6EjI...
85,85,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.9588372,-60.6456409",-32.958837,...,63.0,NaN,NaN,1.0,1.0,NaN,http://www.properati.com.ar/15bz1_ve...,Corrientes 1900 – 1º Piso Departamen...,Corrientes 1900 – 1º Piso 1 dormito...,https://thumbs4.properati.com/8/Gc4B...


In [17]:
df_propierati.groupby("property_type")['property_type'].size()

property_type
PH            5751
apartment    71065
house        40268
store         4136
Name: property_type, dtype: int64

In [18]:
df_propierati.groupby(['state_name','place_name']).size()

state_name                place_name              
Bs.As. G.B.A. Zona Norte   Country Maschwitz Club       3
                           los alamos                   1
                          Acacias Blancas              34
                          Acassuso                    242
                          Albanueva Barrio Cerrado     26
                                                     ... 
Tucumán                   Tafí Viejo                   16
                          Tafí del Valle               22
                          Tucumán                      77
                          Villa Belgrano                1
                          Yerba Buena                 191
Length: 1098, dtype: int64

In [19]:
ba_pn_mask = df_propierati.state_name.isin(['Capital Federal', 'Santa Fe'])
ba_pn_mask.value_counts()

False    78732
True     42488
Name: state_name, dtype: int64

In [20]:
df_propierati.loc[ba_pn_mask, 'state_name'].value_counts()

Capital Federal    32316
Santa Fe           10172
Name: state_name, dtype: int64

In [21]:
df_propierati.loc[ba_pn_mask, :].groupby(['state_name','place_name']).size()

state_name       place_name        
Capital Federal  Abasto                 113
                 Agronomía               41
                 Almagro               1165
                 Balvanera              667
                 Barracas               425
                                       ... 
Santa Fe         Susana                   1
                 Timbúes                  3
                 Venado Tuerto            1
                 Villa Constitución      20
                 Álvarez                  1
Length: 138, dtype: int64

In [22]:
np.round(df_propierati.pivot_table(values=['price_aprox_usd','surface_covered_in_m2'], index='state_name', aggfunc='mean'), 2).head(10)

,price_aprox_usd,surface_covered_in_m2
state_name,,
Bs.As. G.B.A. Zona Norte,324317.12,179.88
Bs.As. G.B.A. Zona Oeste,167545.65,129.84
Bs.As. G.B.A. Zona Sur,198555.72,149.39
Buenos Aires Costa Atlántica,143430.99,117.85
Buenos Aires Interior,183250.00,163.40
Capital Federal,269603.18,97.79
Catamarca,725471.71,260.76
Chaco,178583.55,182.27
Chubut,342338.18,279.18


In [23]:
patt_bsas = '^Buenos Aires|Bs\.As\.+'
reg_bsas = re.compile(patt_bsas)
#df_propierati.place_name.str.contains(reg_bsas)

In [24]:
df_propierati.description.head(1)

0    2 AMBIENTES TIPO CASA PLANTA BAJA PO...
Name: description, dtype: object

In [25]:
print(len(df_propierati.rooms), df_propierati.rooms.isnull().sum(), df_propierati.rooms.notnull().sum())
print(df_propierati.rooms.isnull().sum() + df_propierati.rooms.notnull().sum())
df_propierati.rooms.value_counts()
df_propierati.rooms.describe()

121220 73830 47390
121220


count    47390.000000
mean         3.080840
std          1.860773
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max         32.000000
Name: rooms, dtype: float64

In [26]:
df_propierati.state_name.value_counts()
df_propierati.state_name.isnull().sum()
df_propierati.state_name.notnull().sum()

121220

In [ ]:
df_propierati